# Erase Schema
This notebook drops all user-defined functions, tables, and views from a schema and then removes the schema. Provide the catalog and schema names with the widgets below and run all cells.

In [0]:
try:
    dbutils.widgets.text("1.catalog", "catalog")
    dbutils.widgets.text("2.schema", "schema")
    catalog = dbutils.widgets.get("1.catalog")
    schema = dbutils.widgets.get("2.schema")
except NameError:
    catalog = "do_not_use"
    schema = "do_not_use"

In [0]:
print(f"USE CATALOG `{catalog}`;")
print(f"USE SCHEMA `{schema}`;")

# Drop functions sequentially
funcs_df = spark.sql(f"SHOW USER FUNCTIONS IN `{catalog}`.`{schema}`")
for f in funcs_df.collect():
    func_name = f[0]
    print(f"DROP FUNCTION IF EXISTS `{catalog}`.`{schema}`.`{func_name}`;")

# Drop tables sequentially
tables_df = spark.sql(f"SHOW TABLES IN `{catalog}`.`{schema}`")
for t in tables_df.collect():
    desc = spark.sql(f"DESCRIBE EXTENDED `{catalog}`.`{schema}`.`{t.tableName}`").collect()
    t_type = next((row.data_type for row in desc if str(row.col_name).upper() == 'TYPE'), '').upper().replace('_', ' ')
    if t_type == 'VIEW':
        print(f"DROP VIEW IF EXISTS `{catalog}`.`{schema}`.`{t.tableName}`;")
    elif t_type == 'MATERIALIZED_VIEW':
        print(f"DROP MATERIALIZED VIEW IF EXISTS `{catalog}`.`{schema}`.`{t.tableName}`;")
    else:
        print(f"DROP TABLE IF EXISTS `{catalog}`.`{schema}`.`{t.tableName}`;")

# Drop external volumes
vol_query = f"""SELECT volume_name FROM system.information_schema.volumes\nWHERE volume_catalog = '{catalog}' AND volume_schema = '{schema}' AND volume_type = 'EXTERNAL'"""
vol_df = spark.sql(vol_query)
for row in vol_df.collect():
    print(f"DROP VOLUME IF EXISTS `{catalog}`.`{schema}`.`{row.volume_name}`;")

print(f"DROP SCHEMA IF EXISTS `{catalog}`.`{schema}`;")

In [0]:
print(f"Schema `{schema}` in catalog `{catalog}` cleanup complete.")